#### run global setup

In [ ]:
try:
    with open("../global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

#### run local setup

In [ ]:
%matplotlib inline

import gym
from gym import logger

logger.set_level(logger.ERROR)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm

from src.rl.RandomAgent import RandomAgent
from src.rl.util import run_episode
from src.rl.TabularQAgent import TabularQAgent
from src.rl.NeuralQAgent import NeuralQAgent

## CartPole Challenge

In this notebook we're going to look at a more difficult setting. Here, we have a pole balancing on top of a cart. 
The cart can move left or right, and the goal is to keep the pole balanced on top, without moving too far away from the starting position.  

We can render the setting like this:

In [ ]:
env = gym.make('CartPole-v1')
env.reset()
plt.imshow(env.render(mode='rgb_array'))
env.close()

Let us consider how the state space for this task could be designed. Obviously we're going to need the position of the cart and the angle of the pole to be part of the state space, since these are the things we're wanting to control. Additionally, due to assuming what's called the Markov property, we'll need to include the velocity of the cart and the pole. It might not be clear why we need to include the velocities, but for now you should just accept that this is the case. We're not going to discuss what the Markov property means in this course.

The result is a 4-dimensional state space -- the grid world from the previous exercise had a 2-dimensional state space. However, there is more to the picture than the dimensionality of the space. Whereas the grid world had around $4*12 = 48$ distinct states, each dimension in the 4-dimensional state space is in theory a real number. This leads to an effectively infinite number of distinct states that the pole and cart could be in -- even the smallest change in angle or position is distinct.

First, let's see the performance of a random policy.

In [ ]:
env = gym.make('CartPole-v1')
agent = RandomAgent()
run_episode(env, agent, render=True)
dist = [run_episode(env, agent) for _ in range(1000)]
sns.distplot(dist, kde=False)

Now, let's see how our tabular Q-learning agent that we used to solve the grid world task works. First, a learning curve.

In [ ]:
agent = TabularQAgent(0.1, 0.5, 0.99)

def run_experiment(env, agent, epsilon_decay, n_episodes) -> list:
    rewards = []
    for i in tqdm(range(n_episodes)):
        sum_r = run_episode(env, agent, learn=True)
        rewards.append(sum_r)
        agent.epsilon *= epsilon_decay
    agent.epsilon = 0
    sum_r = run_episode(env, agent)
    print('Trained for ', n_episodes, ' episodes. Last episode achieved a reward of ', sum_r)     
    #env.render(mode='path', ss=ss)
    return rewards


#run_episode(env, agent, learn=True)
rewards = run_experiment(env, agent, 0.99, 1000)
sns.tsplot(rewards)
sns.despine()

And the corresponding histogram.

In [ ]:
sns.distplot(rewards, kde=False)

It didn't seem to learn anything -- and the reason is the size of the state space. Intuitively, since we have so many states, we're unlikely to ever end up in the same state twice -- and so we'll never learn anything by storing our knowledge in a lookup table. What we can do instead, is to use *function approximation*. Instead of a lookup table, we're going to learn a much simpler, continuous function that maps a state to the value we'd otherwise store in the table. Since this function is much simpler, there is no way it's going to return the exact same values as our table would -- but we can hope that it's going to be good enough.

Training this agent is going to be significantly slower than what we've seen previously. The reason is that, as the agent gets better, each episode takes significantly longer to end -- and because the problem is harder, we're going to need more training data, and more time to make use of that data.

In [ ]:
agent = NeuralQAgent(4, env.action_space.n, alpha=0.001, gamma=0.95, epsilon=1.0)
rewards = run_experiment(env, agent, 0.995, 500)
sns.tsplot(rewards)
sns.despine()

That learning curve should look significantly better than the tabular one. It might be unstable and jump up and down, but at least it is learning. Let's look at a histogram.

In [ ]:
dist = [run_episode(env, agent) for _ in tqdm(range(200))]
sns.distplot(dist, kde=False)
print("Mean neural agent reward: ", np.mean(dist))

Significantly better, often reaching the maximal reward of 500 in this environment. We can even render the policy in action -- try running it a few times and see how it behaves.

In [ ]:
env = gym.make('CartPole-v1')
run_episode(env, agent, render=True)